In [32]:
import os 
root = os.getcwd()
root 

'd:\\IT\\GITHUB\\Hutech-AI-Challenge\\PatternFinding'

In [33]:
!nvidia-smi

Thu Apr 10 01:29:24 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.94                 Driver Version: 560.94         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1650 Ti   WDDM  |   00000000:01:00.0  On |                  N/A |
| N/A   52C    P8              4W /   50W |    1435MiB /   4096MiB |     19%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [34]:
import torch 
print(torch.__version__)
print(torch.cuda.is_available())

1.13.1+cu117
True


# Pytorch DataLoader


In [ ]:
# !pip install opencv-python pillow torchvision

In [31]:
import cv2
import numpy as np
from PIL import Image

def apply_clahe_pil(image_pil):
    """Apply CLAHE để normalize độ tương phản của ảnh đầu vào.
    Args:
        image_pil (PIL.Image): Ảnh đầu vào dưới dạng đối tượng PIL."""
    img = np.array(image_pil)

    # Chuyển sang LAB
    lab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
    l, a, b = cv2.split(lab)

    # Áp dụng CLAHE lên kênh L
    clahe = cv2.createCLAHE(clipLimit=0.5, tileGridSize=(4, 4))
    cl = clahe.apply(l)

    # Hợp lại và chuyển về RGB
    merged = cv2.merge((cl, a, b))
    final = cv2.cvtColor(merged, cv2.COLOR_LAB2RGB)

    return Image.fromarray(final)


In [16]:
import torchvision.transforms as transforms

class CLAHETransform:
    def __call__(self, img):
        return apply_clahe_pil(img)


In [65]:
import torchvision.transforms as transforms

train_transform = transforms.Compose([
    # CLAHETransform(),                            # (1) Chuẩn hoá tương phản cục bộ
    transforms.RandomHorizontalFlip(p=0.5),      # (2) Lật ngang
    transforms.RandomVerticalFlip(p=0.2),        # (3) Lật dọc (ít hơn)
    transforms.RandomRotation(15),               # (4) Xoay ±15 độ
    # transforms.ColorJitter(brightness=0.2,       # (5) Điều chỉnh độ sáng
    #                        contrast=0.2,
    #                        saturation=0.2,
    #                        hue=0.02),
    transforms.RandomAffine(degrees=0,
                            translate=(0.1, 0.1),  # (6) Dịch vị trí
                            scale=(0.9, 1.1)),     # (7) Zoom in/out nhẹ

    transforms.ToTensor(),                        # (8) Chuyển về Tensor
    # transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
    transforms.Normalize(mean=[0.4397, 0.3948, 0.3603], std=[0.1841, 0.1777, 0.1702])  #train
])


val_transform = transforms.Compose([
    CLAHETransform(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])


In [66]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

train_dataset = ImageFolder(root='D:\\IT\\GITHUB\\Hutech-AI-Challenge\\data\\train', transform=train_transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)


In [67]:
from torch.utils.data import Dataset, DataLoader
import os
from PIL import Image

class FlatImageDataset(Dataset):
    """Dataset for loading images from a flat directory structure (no class subdirectories)."""
    
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = [
            os.path.join(root_dir, filename) 
            for filename in os.listdir(root_dir) 
            if filename.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))
        ]
        
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert('RGB')  # Ensure RGB format
        
        if self.transform:
            image = self.transform(image)
            
        # Return image with a dummy label (0)
        return image, 0
        
# Replace the test dataset and loader
test_path = "D:\\IT\\GITHUB\\Hutech-AI-Challenge\\data\\test"
test_dataset = FlatImageDataset(root_dir=test_path, transform=val_transform)
test_loader = DataLoader(
    test_dataset, 
    batch_size=32, 
    shuffle=False, 
    num_workers=0,  # Start with 0 to avoid worker errors
    pin_memory=torch.cuda.is_available()
)

print(f"Loaded {len(test_dataset)} test images")

Loaded 200 test images


In [61]:
print(test_loader.dataset[0][0].shape)

torch.Size([3, 32, 32])


In [68]:
# "Ví dụ về cách train mô hình với Dataloader"

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.models import resnet18

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Model
model = resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 4)  # 4 loại nấm
model = model.to(device)

# Loss và Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Training loop
for epoch in range(10):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/10] - Loss: {running_loss/len(train_loader):.4f}")


d:\Anaconda\envs\Research\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\Anaconda\envs\Research\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/10] - Loss: 0.9480
Epoch [2/10] - Loss: 0.5162
Epoch [3/10] - Loss: 0.3277
Epoch [4/10] - Loss: 0.2908
Epoch [5/10] - Loss: 0.1802
Epoch [6/10] - Loss: 0.1773
Epoch [7/10] - Loss: 0.1396
Epoch [8/10] - Loss: 0.1396
Epoch [9/10] - Loss: 0.1516
Epoch [10/10] - Loss: 0.0999


In [63]:
model.eval()
predictions = []

with torch.no_grad():
    for images, _ in test_loader:  # Assuming test_loader doesn't have labels
        images = images.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        predictions.extend(predicted.cpu().numpy())

print(f"Predictions: {predictions}")

Predictions: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 2, 2, 2, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 3, 0, 3, 3, 3, 3, 0, 3, 3, 0, 0, 0, 0, 0, 3, 0, 3, 3, 3, 3, 3, 3, 0, 3, 3, 0, 3, 3, 3, 0, 0, 3, 0, 0, 3, 3, 0, 0, 3, 3, 3, 3, 0, 3, 0, 0, 0, 0, 0, 3, 3, 0, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 3, 3, 0, 0, 0, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 2, 2, 0, 2, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2]


In [ ]:
#Test-Time Augmentation - Augment trong lúc test rồi lấy major voting
import torchvision.transforms as transforms

# Các TTA transforms áp dụng cho mỗi ảnh
tta_transforms = [
    transforms.Compose([transforms.ToTensor()]),
    transforms.Compose([transforms.RandomHorizontalFlip(p=1.0), transforms.ToTensor()]),
    transforms.Compose([transforms.RandomRotation(degrees=10), transforms.ToTensor()]),
    transforms.Compose([transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)), transforms.ToTensor()])
]


In [41]:
import torch
from tqdm import tqdm

def predict_loader_with_tta(model, dataloader, tta_transforms, device):
    model.eval()
    all_preds = []

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Predicting with TTA"):
            images, _ = batch  # Giả sử test set không có nhãn
            batch_preds = []

            for t in tta_transforms:
                augmented_images = torch.stack([t(transforms.ToPILImage()(img.cpu())) for img in images])
                augmented_images = augmented_images.to(device)
                outputs = model(augmented_images)
                probs = torch.softmax(outputs, dim=1)
                batch_preds.append(probs)

            # Trung bình xác suất qua các phiên bản TTA
            avg_preds = torch.mean(torch.stack(batch_preds), dim=0)
            pred_labels = torch.argmax(avg_preds, dim=1)
            all_preds.extend(pred_labels.cpu().numpy())

    return all_preds


In [ ]:
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

test_dataset = FlatImageDataset(root_dir=test_path, transform=val_transform)
test_loader = DataLoader(
    test_dataset, 
    batch_size=32, 
    shuffle=False, 
    num_workers=0,  # Start with 0 to avoid worker errors
    pin_memory=torch.cuda.is_available()
)

# Thiết bị
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Dự đoán có TTA
predictions = predict_loader_with_tta(model, test_loader, tta_transforms, device)


Predicting with TTA: 100%|██████████| 7/7 [00:00<00:00,  8.63it/s]


In [ ]:
print(predictions)

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 3,
 0,
 2,
 2,
 2,
 0,
 0,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 3,
 0,
 3,
 3,
 3,
 3,
 0,
 3,
 3,
 0,
 0,
 0,
 0,
 0,
 3,
 0,
 3,
 3,
 3,
 3,
 3,
 3,
 0,
 3,
 3,
 0,
 3,
 3,
 3,
 0,
 0,
 3,
 0,
 0,
 3,
 3,
 0,
 0,
 3,
 3,
 3,
 3,
 0,
 3,
 0,
 0,
 0,
 0,
 0,
 3,
 3,
 0,
 3,
 3,
 3,
 3,
 3,
 0,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 0,
 0,
 0,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 2,
 2,
 2]

# Build

In [12]:
# Calculate mean and std for normalization
import torch
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader
import numpy as np
from tqdm import tqdm
import os

def calculate_dataset_statistics(dataset_path, batch_size=64, num_workers=2):
    """Calculate mean and std of a dataset for normalization."""
    try:
        # Verify path exists
        if not os.path.exists(dataset_path):
            raise FileNotFoundError(f"Dataset path not found: {dataset_path}")
            
        # Create dataset and loader
        dataset = ImageFolder(dataset_path, transform=transforms.ToTensor())
        loader = DataLoader(
            dataset, 
            batch_size=batch_size, 
            shuffle=False, 
            num_workers=num_workers,
            pin_memory=torch.cuda.is_available()
        )
        
        print(f"Processing {len(dataset)} images from {dataset_path}")
        
        # Initialize statistics
        mean = 0.0
        std = 0.0
        nb_samples = 0
        
        # Process batches with progress bar
        for data, _ in tqdm(loader, desc="Calculating statistics"):
            batch_samples = data.size(0)
            data = data.view(batch_samples, data.size(1), -1)
            mean += data.mean(2).sum(0)
            std += data.std(2).sum(0)
            nb_samples += batch_samples
        
        # Calculate final statistics
        mean /= nb_samples
        std /= nb_samples
        
        return mean.numpy(), std.numpy()
    
    except Exception as e:
        print(f"Error calculating dataset statistics: {str(e)}")
        return None, None

# Calculate for train dataset
train_path = "D:\\IT\\GITHUB\\Hutech-AI-Challenge\\data\\train"
train_mean, train_std = calculate_dataset_statistics(train_path)
print(f"Train Mean: {train_mean}")
print(f"Train Std: {train_std}")


Processing 1200 images from D:\IT\GITHUB\Hutech-AI-Challenge\data\train


Calculating statistics: 100%|██████████| 19/19 [00:04<00:00,  3.92it/s]

Train Mean: [0.43967322 0.3947982  0.36026132]
Train Std: [0.18411897 0.1776691  0.17019545]


In [6]:
import os
from PIL import Image
import shutil
from tqdm import tqdm

INPUT_FOLDER = "D:\\IT\\GITHUB\\Hutech-AI-Challenge\\data\\train"
OUTPUT_FOLDER = "D:\\IT\\GITHUB\\Hutech-AI-Challenge\\data\\train_CLAHE"

# Clear output directory if needed
if os.path.exists(OUTPUT_FOLDER):
    shutil.rmtree(OUTPUT_FOLDER)
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# Get all image files
image_files = []
for root, _, files in os.walk(INPUT_FOLDER):
    for file in files:
        if file.lower().endswith((".jpg", ".png", ".jpeg")):
            image_files.append((root, file))

print(f"Found {len(image_files)} images to process")

# Process images with progress bar
for root, file in tqdm(image_files, desc="Processing images"):
    try:
        # Get relative path to maintain folder structure
        rel_path = os.path.relpath(root, INPUT_FOLDER)
        if rel_path != ".":
            output_dir = os.path.join(OUTPUT_FOLDER, rel_path)
            os.makedirs(output_dir, exist_ok=True)
        else:
            output_dir = OUTPUT_FOLDER
            
        # Process image
        path = os.path.join(root, file)
        image = Image.open(path).convert("RGB")  # Ensure RGB mode
        enhanced = apply_clahe(image)
        
        # Save processed image
        save_path = os.path.join(output_dir, file)
        enhanced.save(save_path)
        
    except Exception as e:
        print(f"Error processing {file}: {str(e)}")

Found 1200 images to process


Processing images: 100%|██████████| 1200/1200 [00:02<00:00, 513.07it/s]
